In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import time
import json5
import json

In [2]:
_ = load_dotenv()
openai_client = OpenAI()

GPT3_MODEL = "gpt-3.5-turbo-1106"
GPT4_MODEL = "gpt-4-1106-preview"

In [4]:
input_data = """
[
  { "name": "Sachin", "team": "India" },
  { "name": "Sourav", "team": "India" },
  { "name": "Lara", "team": "West Indies" }
]
"""

output_data = """
[
 { "team": "India", "playerCount": 2 },
 { "team": "West Indies", "playerCount": 1 }
]
"""

In [5]:
assistant_instructions = """You are a MongoDB expert 
with great expertise in writing MongoDB queries for any given data 
to produce an expected output."""

user_prompt = f"""Your task is to write a MongoDB Query, 
specifically an aggregation pipeline that would produce 
the expected output for the given input.

You will always return a JSON response with the following fields.
```
mongoDBQuery: The MongoDB aggregation pipeline to produce 
the expected output for a given input. This field corresponds 
to just the list of stages in the aggregation pipeline 
and shouldn't contain the "db.collection.aggregate" prefix.
    
queryExplanation: A detailed explanation for the query 
that was returned.
```
    
Input data: {input_data} 
Expected output data: {output_data}
"""

In [8]:
assistant = openai_client.beta.assistants.create(
    name="MongoDB assistant",
    instructions=assistant_instructions,
    model=GPT3_MODEL
)

MONGO_DB_ASSISTANT_ID = assistant.id
MONGO_DB_ASSISTANT_ID

'asst_hDT9ZCVxasrIqDtq0QhzgKUy'

In [9]:
thread = openai_client.beta.threads.create()

message = openai_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_prompt
)

In [10]:
run = openai_client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [11]:
for i in range(60):
    try:
        updated_run = openai_client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )
        # Check if the status indicates completion
        if updated_run.status == "completed":
            messages = openai_client.beta.threads.messages.list(
                thread_id=thread.id
            )
            print(f"Assistants Response: {messages.data[0].content[0].text.value}")
            break
            except Exception as e:
                print(f"Error: {str(e)}. Trying again...")
            finally:
                time.sleep(2)
        else:        
            # If max_attempts reached without completion, then assistant call timed out
            print("Timeout: Assistant didn't respond in time. Please try again.")

TabError: inconsistent use of tabs and spaces in indentation (825533230.py, line 16)